# Charts with XPlot
Charts can be rendered using [Xplot.Plotly](https://fslab.org/XPlot/). 
We will cover some example on how to use XPlot in a notebook with the .NET Kernel.

# Rendering Scatter plots
One of the most commonly used type of chart to explore data set. Use the type `Scatter`.

In [3]:
$openSeries = New-PlotlyTrace -TraceType Scatter -Name Open
$openSeries.x = @(1, 2, 3, 4)
$openSeries.y = @(10, 15, 13, 17)

$closeSeries = New-PlotlyTrace -TraceType Scatter -Name Close
$closeSeries.x = @(2, 3, 4, 5)
$closeSeries.y = @(16, 5, 11, 9)

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

Let's change it to be markers style, so more like a scatter plot.

In [4]:
$openSeries.mode = "markers";
$closeSeries.mode = "markers";

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

`Scatter` can also produce polar charts by setting the radial property `r` and angular proeprty `t`

In [5]:
$openSeries = New-PlotlyTrace -TraceType Scatter -Name Open
$openSeries.r = @(1, 2, 3, 4)
$openSeries.t = @(45, 100, 150, 290)

$closeSeries = New-PlotlyTrace -TraceType Scatter -Name Close
$closeSeries.r = @(2, 3, 4, 5)
$closeSeries.t = @(16, 45, 118, 90)

$layout = New-PlotlyLayout
$layout.orientation = -90

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close" -Layout $layout
$chart | Out-Display

## Large scatter plots and performance
It is not uncommon to have scatter plots with a large dataset, it is a common scenario at the beginning of a data exploration process. Using the default `svg` based rendering will create performace issues as the dom will become very large.
We can then use `web-gl` support to address the problem.

In [6]:
#!time
# var series = new Graph.Scattergl[10];

# for(var i = 0; i < series.Length; i++){
#     series[i] =  new Graph.Scattergl
#     {
#         name = $"Series {i}",
#         mode = "markers",
#         x = Enumerable.Range(0,100000).Select(_ => generator.Next(-200, 200) * 1000 * generator.Next(-2000, 2000)),
#         y = Enumerable.Range(0,100000).Select(_ => generator.Next(-200, 200) * 1000 * generator.Next(-2000, 2000))
#     };
# }

# chart = Chart.Plot(series);
# chart.WithTitle("Large Dataset");
# display(chart);

$x = Get-Random -Count 100000 -Minimum -100000 -Maximum 100000
$y = Get-Random -Count 100000 -Minimum -100000 -Maximum 100000
$series = 1..10 | ForEach-Object {
    $trace = New-PlotlyTrace -TraceType Scattergl -Name "Series $_"
    $trace.mode = "markers"
    $trace.x = $x
    $trace.y = $y

    $trace
}

New-PlotlyChart -Title "Large Dataset" -Trace $series | Out-Display

Wall time: 26877.2165ms

Can provide custom marker `colour`, `size` and `colorscale` to display even more information to the user.

In [ ]:
for(var i = 0; i < series.Length; i++){
    var sizes = Enumerable.Range(0,100).Select(_ => (generator.NextDouble() < 0.75) ? generator.Next(1, 5) : generator.Next(10, 15)).ToArray();
    var temperatures = sizes.Select(t => (t * 10) - 100);
    
    series[i].x = Enumerable.Range(0,100).Select(_ => generator.Next(-200, 200) * 1000 * generator.Next(-2000, 2000));
    series[i].y = Enumerable.Range(0,100).Select(_ => generator.Next(-200, 200) * 1000 * generator.Next(-2000, 2000));
    series[i].marker = new Graph.Marker{        
        size = sizes,
        color = temperatures,
        colorscale = "hot"    
    };
}

chart = Chart.Plot(series);
chart.WithTitle("Size and Colour");
display(chart);

Plotly pvoides some additional `color scales` to use.

In [ ]:
for(var i = 0; i < series.Length; i++) {
    series[i].marker.colorscale = "Viridis";
}

chart = Chart.Plot(series);
chart.WithTitle("Viridis scale");
display(chart);

for(var i = 0; i < series.Length; i++) {
    series[i].marker.colorscale = "Hot";
}

chart = Chart.Plot(series);
chart.WithTitle("Hot scale");
display(chart);

for(var i = 0; i < series.Length; i++) {
    series[i].marker.colorscale = "Jet";
}

chart = Chart.Plot(series);
chart.WithTitle("Jet scale");
display(chart);

# Rendering Histograms
Let's have a look at using histograms, the next cell sets up some generators.

In [7]:
$count = 20
$dates = 1..$count | ForEach-Object { (Get-Date).AddMinutes((Get-Random -Minimum $_ -Maximum ($_+30))) }

Now let's define histogram traces:

In [8]:
$openByTime = New-PlotlyTrace -TraceType Histogram -Name Open
$openByTime.x = $dates
$openByTime.y = Get-Random -Count $count -Minimum 0 -Maximum 200

$closeByTime = New-PlotlyTrace -TraceType Histogram -Name Close
$closeByTime.x = $dates
$closeByTime.y = Get-Random -Count $count -Minimum 0 -Maximum 200

New-PlotlyChart -Trace @($openByTime, $closeByTime) | Out-Display

The Histogram generator will automatically count the number of items per bin. 

Setting `histfunc` to `"sum"` we can now add up all the values contained in each bin.
Note that we are creatng bin using the `x` data point and we are using bydefault autobinx

In [9]:
$openByTime.histfunc = "sum"
$closeByTime.histfunc = "sum"

New-PlotlyChart -Trace @($openByTime, $closeByTime) | Out-Display

# Area chart and Polar Area chart

By populating hte property `fill` of a `Scatter` trace the chart will render as area chart.

Here is set to `"tozeroy"` which will create a fill zone underneath the line reachin to the 0 of the y axis.

In [10]:
$openSeries = New-PlotlyTrace -TraceType Scatter -Name Open
$openSeries.x = @(1, 2, 3, 4)
$openSeries.y = @(10, 15, 13, 17)
$openSeries.fill = "tozeroy"
$openSeries.mode = "lines"

$closeSeries = New-PlotlyTrace -TraceType Scatter -Name Close
$closeSeries.x = @(1, 2, 3, 4)
$closeSeries.y = @(3, 5, 11, 9)
$closeSeries.fill = "tozeroy"
$closeSeries.mode = "lines"

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

With one `fill` set to `"tonexty"` the cahrt will fill the aread between traces.

In [11]:
$openSeries.fill = $null;
$closeSeries.fill = "tonexty";

$chart = @($openSeries, $closeSeries) | New-PlotlyChart -Title "Open vs Close"
Out-Display $chart

Using `Area` traces we can generate radial area chart. In this example we are using cardinal points to xpress angular values.
The array `{"North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W"}` will be autoimatically translated to angular values.

In [12]:
$areaTrace1 = New-PlotlyTrace -TraceType Area
$areaTrace1.r = @(77.5, 72.5, 70.0, 45.0, 22.5, 42.5, 40.0, 62.5)
$areaTrace1.t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
$areaTrace1.name = "11-14 m/s"
$areaTrace1.marker = New-PlotlyObject -ObjectType Marker
$areaTrace1.marker.color = "rgb(106,81,163)"

$areaTrace2 = New-PlotlyTrace -TraceType Area
$areaTrace2.r = @(57.49999999999999, 50.0, 45.0, 35.0, 20.0, 22.5, 37.5, 55.00000000000001)
$areaTrace2.t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
$areaTrace2.name = "8-11 m/s"
$areaTrace2.marker = New-PlotlyObject -ObjectType Marker
$areaTrace2.marker.color = "rgb(158,154,200)"

$areaTrace3 = New-PlotlyTrace -TraceType Area
$areaTrace3.r = @(40.0, 30.0, 30.0, 35.0, 7.5, 7.5, 32.5, 40.0)
$areaTrace3.t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
$areaTrace3.name = "5-8 m/s"
$areaTrace3.marker = New-PlotlyObject -ObjectType Marker
$areaTrace3.marker.color = "rgb(203,201,226)"

$areaTrace4 = New-PlotlyTrace -TraceType Area
$areaTrace4.r = @(20.0, 7.5, 15.0, 22.5, 2.5, 2.5, 12.5, 22.5)
$areaTrace4.t = @("North", "N-E", "East", "S-E", "South", "S-W", "West", "N-W")
$areaTrace4.name = "< 5 m/s"
$areaTrace4.marker = New-PlotlyObject -ObjectType Marker
$areaTrace4.marker.color = "rgb(242,240,247)"

$areaLayout = New-PlotlyLayout -Title "Wind Speed Distribution in Laurel, NE"
$areaLayout.font = New-PlotlyObject -ObjectType Font
$areaLayout.font.size = 16
$areaLayout.legend = New-PlotlyObject -ObjectType Legend
$areaLayout.legend.font = New-PlotlyObject -ObjectType Font
$areaLayout.legend.font.size = 16
$areaLayout.radialaxis = New-PlotlyObject -ObjectType Radialaxis
$areaLayout.radialaxis.ticksuffix = "%"
$areaLayout.orientation = 270

New-PlotlyChart -Layout $areaLayout -Trace @($areaTrace1, $areaTrace2, $areaTrace3, $areaTrace4) | Out-Display